In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw, ImageFont, ImageOps
import cv2
import numpy as np
import os
import subprocess
import tempfile
import platform
from datetime import datetime
import math
import ipywidgets as widgets
from IPython.display import display


class MacOSFileBrowser:
    """
    A utility for selecting files and directories on macOS using AppleScript to
    display native file dialogs, avoiding TKinter-related issues.
    """

    @staticmethod
    def select_file(title="Select a file", file_types=None, initial_folder=None):
        """
        Show a native macOS file selection dialog.
        
        Args:
            title: Dialog title
            file_types: List of file extensions (e.g., ["jpg", "png"])
            initial_folder: Initial directory to show
            
        Returns:
            Selected file path or None if canceled
        """
        if platform.system() != "Darwin":
            return None
        
        # Build the AppleScript
        applescript = [
            'set theFile to choose file',
            f'with prompt "{title}"'
        ]
        
        # Add file type filter if provided
        if file_types and len(file_types) > 0:
            extensions = ', '.join([f'"{ext}"' for ext in file_types])
            applescript.append(f'of type {{{extensions}}}')
        
        # Add initial folder if provided
        if initial_folder:
            applescript.append(f'default location "{initial_folder}"')
        
        # Finalize script
        applescript.append('return POSIX path of theFile')
        applescript_cmd = ' '.join(applescript)
        
        # Run AppleScript
        try:
            result = subprocess.run(['osascript', '-e', applescript_cmd], 
                                    capture_output=True, text=True, check=True)
            # Strip newline from the result
            return result.stdout.strip()
        except subprocess.CalledProcessError:
            # User canceled or error occurred
            return None
        
    @staticmethod
    def select_folder(title="Select a folder", initial_folder=None):
        """
        Show a native macOS folder selection dialog.
        
        Args:
            title: Dialog title
            initial_folder: Initial directory to show
            
        Returns:
            Selected folder path or None if canceled
        """
        if platform.system() != "Darwin":
            return None
        
        # Build the AppleScript
        applescript = [
            'set theFolder to choose folder',
            f'with prompt "{title}"'
        ]
        
        # Add initial folder if provided
        if initial_folder:
            applescript.append(f'default location "{initial_folder}"')
        
        # Finalize script
        applescript.append('return POSIX path of theFolder')
        applescript_cmd = ' '.join(applescript)
        
        # Run AppleScript
        try:
            result = subprocess.run(['osascript', '-e', applescript_cmd], 
                                    capture_output=True, text=True, check=True)
            # Strip newline from the result
            return result.stdout.strip()
        except subprocess.CalledProcessError:
            # User canceled or error occurred
            return None
            
    @staticmethod
    def save_file(title="Save file", default_name=None, file_types=None, initial_folder=None):
        """
        Show a native macOS save file dialog.
        
        Args:
            title: Dialog title
            default_name: Default filename
            file_types: List of file extensions (e.g., ["jpg", "png"])
            initial_folder: Initial directory to show
            
        Returns:
            Selected save path or None if canceled
        """
        if platform.system() != "Darwin":
            return None
        
        # Build the AppleScript
        applescript = [
            'set theFile to choose file name',
            f'with prompt "{title}"'
        ]
        
        # Add default name if provided
        if default_name:
            applescript.append(f'default name "{default_name}"')
            
        # Add initial folder if provided
        if initial_folder:
            applescript.append(f'default location "{initial_folder}"')
        
        # Finalize script
        applescript.append('return POSIX path of theFile')
        applescript_cmd = ' '.join(applescript)
        
        # Run AppleScript
        try:
            result = subprocess.run(['osascript', '-e', applescript_cmd], 
                                    capture_output=True, text=True, check=True)
            # Strip newline from the result
            path = result.stdout.strip()
            
            # Add extension if it's not already there and file_types is provided
            if file_types and len(file_types) > 0 and not any(path.lower().endswith(f".{ext.lower()}") for ext in file_types):
                path = f"{path}.{file_types[0]}"
                
            return path
        except subprocess.CalledProcessError:
            # User canceled or error occurred
            return None

    @staticmethod
    def is_macos():
        """Check if the current platform is macOS"""
        return platform.system() == "Darwin"


class AnimationToContactSheetApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Animation to Contact Sheet Converter")
        self.root.geometry("900x700")
        
        # Variables for storing the animation
        self.animation_frames = []
        self.current_frame_index = 0
        self.animation_path = ""
        self.animation_type = ""  # "gif" or "video"
        
        # Paper size settings
        self.paper_size = tk.StringVar(value="letter")
        self.paper_orientation = tk.StringVar(value="landscape")  # Changed default to landscape for wider format

        # Output settings
        self.output_frame_width = tk.IntVar(value=200)  # Increased to 200px default
        self.output_frame_height = 150  # This will become a calculated value, not a UI control
        self.height_display_var = tk.StringVar(value="150px")  # For displaying calculated height
        self.frames_per_row = tk.IntVar(value=4)
        self.registration_mark_size = tk.IntVar(value=20)  # Reduced default to give more space to images
        self.output_dpi = tk.IntVar(value=300)
        self.cmyk_separation = tk.BooleanVar(value=False)  # Default to RGB for most users
        self.auto_multipage = tk.BooleanVar(value=True)  # New variable for auto multi-page
        
        # Image references for display
        self.current_photo = None
        self.preview_photos = []  # For storing preview images to prevent garbage collection
        self.contact_sheet_photo = None  # Reference for contact sheet
        
        # Create the UI
        self.create_ui()

    def create_ui(self):
        # Main frame
        main_frame = ttk.Frame(self.root, padding=10)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Left panel for controls
        control_frame = ttk.LabelFrame(main_frame, text="Controls", padding=10)
        control_frame.pack(side=tk.LEFT, fill=tk.Y, padx=(0, 10))
        
        # Animation input section
        input_frame = ttk.LabelFrame(control_frame, text="Input", padding=10)
        input_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(input_frame, text="Load Animation (GIF/Video)", command=self.load_animation).pack(fill=tk.X)
        
        # Animation navigation
        nav_frame = ttk.LabelFrame(control_frame, text="Navigation", padding=10)
        nav_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Button(nav_frame, text="Previous Frame", command=self.prev_frame).pack(fill=tk.X, pady=(0, 5))
        ttk.Button(nav_frame, text="Next Frame", command=self.next_frame).pack(fill=tk.X)
        
        # Output settings
        output_settings_frame = ttk.LabelFrame(control_frame, text="Output Settings", padding=10)
        output_settings_frame.pack(fill=tk.X, pady=(0, 10))
        
        ttk.Label(output_settings_frame, text="Frame Width (min 200px):").grid(row=0, column=0, sticky=tk.W, pady=2)
        width_spinbox = ttk.Spinbox(output_settings_frame, from_=200, to=1000, textvariable=self.output_frame_width, 
                                command=self.update_frame_height)
        width_spinbox.grid(row=0, column=1, sticky=tk.W, pady=2)
        
        # Display frame height (read-only, calculated automatically)
        ttk.Label(output_settings_frame, text="Frame Height (auto):").grid(row=1, column=0, sticky=tk.W, pady=2)
        self.height_display_var = tk.StringVar(value=f"{self.calculate_frame_height()}px")
        height_label = ttk.Label(output_settings_frame, textvariable=self.height_display_var)
        height_label.grid(row=1, column=1, sticky=tk.W, pady=2)
        
        ttk.Label(output_settings_frame, text="Frames Per Row (Four Recommended):").grid(row=2, column=0, sticky=tk.W, pady=2)
        ttk.Spinbox(output_settings_frame, from_=1, to=10, textvariable=self.frames_per_row).grid(row=2, column=1, sticky=tk.W, pady=2)
        
        ttk.Label(output_settings_frame, text="Registration Mark Size:").grid(row=3, column=0, sticky=tk.W, pady=2)
        ttk.Spinbox(output_settings_frame, from_=5, to=50, textvariable=self.registration_mark_size).grid(row=3, column=1, sticky=tk.W, pady=2)
        
        ttk.Label(output_settings_frame, text="Output DPI:").grid(row=4, column=0, sticky=tk.W, pady=2)
        ttk.Spinbox(output_settings_frame, from_=72, to=600, textvariable=self.output_dpi).grid(row=4, column=1, sticky=tk.W, pady=2)
        
        # Color settings
        ttk.Label(output_settings_frame, text="Color Options:").grid(row=5, column=0, sticky=tk.W, pady=2)
        color_frame = ttk.Frame(output_settings_frame)
        color_frame.grid(row=5, column=1, sticky=tk.W, pady=2)
        
        ttk.Radiobutton(color_frame, text="RGB", variable=self.cmyk_separation, value=False).pack(side=tk.LEFT)
        ttk.Radiobutton(color_frame, text="CMYK", variable=self.cmyk_separation, value=True).pack(side=tk.LEFT)
        
        # Add a tip for the CMYK separation
        ttk.Label(output_settings_frame, text="*Use RGB if CMYK shows inverted colors", 
                font=("Arial", 8), foreground="red").grid(row=6, column=0, columnspan=2, sticky=tk.W)
        
        # Paper size settings
        ttk.Label(output_settings_frame, text="Paper Size:").grid(row=7, column=0, sticky=tk.W, pady=2)
        paper_size_combo = ttk.Combobox(output_settings_frame, textvariable=self.paper_size, 
                                    values=["letter", "tabloid", "a4", "a3"], 
                                    state="readonly", width=10)
        paper_size_combo.grid(row=7, column=1, sticky=tk.W, pady=2)

        ttk.Label(output_settings_frame, text="Orientation:").grid(row=8, column=0, sticky=tk.W, pady=2)
        orientation_combo = ttk.Combobox(output_settings_frame, textvariable=self.paper_orientation, 
                                    values=["portrait", "landscape"], 
                                    state="readonly", width=10)
        orientation_combo.grid(row=8, column=1, sticky=tk.W, pady=2)
        
        # Multi-page option
        ttk.Label(output_settings_frame, text="Auto-split large animations:").grid(row=9, column=0, sticky=tk.W, pady=2)
        ttk.Checkbutton(output_settings_frame, variable=self.auto_multipage).grid(row=9, column=1, sticky=tk.W)
        
        # Add a help text for multi-page
        help_text = "When enabled, animations with many frames will be\nsplit into multiple pages to maintain image quality."
        ttk.Label(output_settings_frame, text=help_text, font=("Arial", 8)).grid(row=10, column=0, columnspan=2, sticky=tk.W, pady=(0, 5))

        # Export buttons
        export_frame = ttk.LabelFrame(control_frame, text="Export", padding=10)
        export_frame.pack(fill=tk.X)
        
        ttk.Button(export_frame, text="Generate Contact Sheet", command=self.generate_contact_sheet).pack(fill=tk.X, pady=(0, 5))
        ttk.Button(export_frame, text="Preview Contact Sheet", command=self.preview_contact_sheet).pack(fill=tk.X)
        
        # Right panel for preview
        preview_frame = ttk.LabelFrame(main_frame, text="Preview", padding=10)
        preview_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        self.canvas = tk.Canvas(preview_frame, bg="white")
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Status bar
        self.status_var = tk.StringVar(value="Ready")
        status_bar = ttk.Label(self.root, textvariable=self.status_var, relief=tk.SUNKEN, anchor=tk.W)

    def calculate_frame_height(self):
        """Calculate frame height based on width and the average aspect ratio of frames"""
        if not self.animation_frames:
            # Default aspect ratio if no frames loaded (4:3)
            return int(self.output_frame_width.get() * 0.75)
        
        # Calculate average aspect ratio from all frames
        total_ratio = 0
        valid_frames = 0
        
        for frame in self.animation_frames:
            if frame.height > 0:  # Avoid division by zero
                total_ratio += frame.width / frame.height
                valid_frames += 1
        
        # Use average ratio, or fallback to 4:3 if no valid frames
        avg_ratio = total_ratio / valid_frames if valid_frames > 0 else 4/3
        
        # Calculate height based on width and aspect ratio
        return int(self.output_frame_width.get() / avg_ratio)

    def update_frame_height(self, event=None):
        """Update the displayed frame height when width changes"""
        calculated_height = self.calculate_frame_height()
        self.height_display_var.set(f"{calculated_height}px")
        
        # Also update the internal frame height variable used for generation
        self.output_frame_height = calculated_height

        
    def load_animation(self, file_path=None):
        """Load animation using a file dialog with improved error handling"""
        try:
            # Reset references to prevent memory issues
            self.current_photo = None
            self.preview_photos = []
            self.contact_sheet_photo = None
            
            if file_path is None:
                # Use tkinter file dialog to select the animation file
                file_path = filedialog.askopenfilename(
                    title="Select Animation File",
                    filetypes=[("GIF/Video Files", "*.gif *.mp4 *.avi *.mov"), ("All Files", "*.*")]
                )
            
            if file_path:
                # Update status
                self.status_var.set(f"Loading {os.path.basename(file_path)}...")
                self.root.update()  # Force update to show status
                
                # Clear existing frames and process the new file
                self.animation_frames = []
                self.current_frame_index = 0
                self.animation_path = file_path
                
                # Process based on file type
                if file_path.lower().endswith('.gif'):
                    self.animation_type = "gif"
                    self.load_gif(file_path)
                else:
                    self.animation_type = "video"
                    self.load_video(file_path)
                    
                # Update frame height based on new frames
                self.update_frame_height()
                    
                # Display first frame if available
                if self.animation_frames:
                    self.current_frame_index = 0
                    self.display_current_frame()
                    self.status_var.set(f"Loaded {len(self.animation_frames)} frames from {os.path.basename(file_path)}")
                else:
                    self.status_var.set("No frames were loaded from the file")
                    messagebox.showerror("Error", "Failed to load animation frames")
        except Exception as e:
            import traceback
            traceback.print_exc()
            self.status_var.set("Error loading animation")
            messagebox.showerror("Error", f"Error during file selection: {str(e)}")



    def process_selected_file(self, file_path):
        """Process the selected animation file"""
        if not file_path or not os.path.exists(file_path):
            messagebox.showerror("Error", "Invalid file path")
            return
            
        self.animation_path = file_path
        self.animation_frames = []
        
        # Process based on file type
        if file_path.lower().endswith('.gif'):
            self.animation_type = "gif"
            self.load_gif(file_path)
        else:
            self.animation_type = "video"
            self.load_video(file_path)
            
        if self.animation_frames:
            self.current_frame_index = 0
            self.display_current_frame()
            self.status_var.set(f"Loaded {len(self.animation_frames)} frames from {os.path.basename(file_path)}")
        else:
            messagebox.showerror("Error", "Failed to load animation frames")
    
    def load_gif(self, file_path):
        """Load all frames from a GIF with improved error handling"""
        try:
            # Open the GIF
            gif = Image.open(file_path)
            
            # Get all frames
            frame_count = 0
            frames_loaded = 0
            
            try:
                # Update status periodically
                self.status_var.set(f"Loading GIF frames...")
                self.root.update()
                
                while True:
                    gif.seek(frame_count)
                    frame = gif.convert('RGBA')
                    self.animation_frames.append(frame.copy())
                    frame_count += 1
                    frames_loaded += 1
                    
                    # Update status every 10 frames
                    if frames_loaded % 10 == 0:
                        self.status_var.set(f"Loaded {frames_loaded} GIF frames...")
                        self.root.update()
                        
            except EOFError:
                # End of frames - this is expected
                pass
            
            # Report success
            print(f"Successfully loaded {len(self.animation_frames)} frames from GIF")
                
        except Exception as e:
            import traceback
            traceback.print_exc()
            raise Exception(f"Failed to load GIF: {str(e)}")
    
    def load_video(self, file_path):
        """Load frames from a video file with improved error handling"""
        try:
            # Open the video
            cap = cv2.VideoCapture(file_path)
            
            if not cap.isOpened():
                raise Exception("Failed to open video file")
            
            # Get video info
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            
            self.status_var.set(f"Loading video: {total_frames} frames at {fps:.2f} FPS")
            self.root.update()
            
            frames_loaded = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Convert OpenCV BGR to PIL RGB
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                pil_frame = Image.fromarray(frame_rgb)
                self.animation_frames.append(pil_frame)
                frames_loaded += 1
                
                # Update status occasionally
                if frames_loaded % 20 == 0:
                    self.status_var.set(f"Loaded {frames_loaded}/{total_frames} video frames...")
                    self.root.update()
            
            # Clean up
            cap.release()
            
            # If no frames were loaded, raise an error
            if len(self.animation_frames) == 0:
                raise Exception("No frames could be extracted from the video")
                
            print(f"Successfully loaded {len(self.animation_frames)} frames from video")
                
        except Exception as e:
            import traceback
            traceback.print_exc()
            raise Exception(f"Failed to load video: {str(e)}")

    
    def display_current_frame(self):
        """Display the current frame with improved error handling"""
        if not self.animation_frames:
            self.canvas.delete("all")
            self.canvas.create_text(
                self.canvas.winfo_width() // 2, 
                self.canvas.winfo_height() // 2,
                text="No frames loaded", 
                fill="black",
                font=("Arial", 14)
            )
            return
            
        try:
            # Get frame with bounds checking
            if 0 <= self.current_frame_index < len(self.animation_frames):
                frame = self.animation_frames[self.current_frame_index]
            else:
                # Reset index if out of bounds
                self.current_frame_index = 0
                frame = self.animation_frames[0]
            
            # Calculate canvas dimensions
            max_display_width = self.canvas.winfo_width() - 20
            max_display_height = self.canvas.winfo_height() - 20
            
            if max_display_width <= 0 or max_display_height <= 0:
                # Canvas not initialized yet, use default size
                max_display_width = 500
                max_display_height = 400
                
            # Calculate resize dimensions
            img_ratio = frame.width / frame.height if frame.height > 0 else 1.0
            
            if frame.width > max_display_width or frame.height > max_display_height:
                if max_display_width / max_display_height > img_ratio:
                    # Height is the limiting factor
                    display_height = max_display_height
                    display_width = int(display_height * img_ratio)
                else:
                    # Width is the limiting factor
                    display_width = max_display_width
                    display_height = int(display_width / img_ratio)
                
                # Ensure minimum display size
                display_width = max(10, display_width)
                display_height = max(10, display_height)
                
                # Resize the frame for display
                display_frame = frame.resize((display_width, display_height), Image.LANCZOS)
            else:
                display_frame = frame
                
            # Create new PhotoImage and store the reference
            self.current_photo = ImageTk.PhotoImage(display_frame)
            
            # Clear canvas and display the image
            self.canvas.delete("all")
            self.canvas.create_image(
                max_display_width // 2, 
                max_display_height // 2, 
                image=self.current_photo,
                anchor=tk.CENTER
            )
            
            # Display frame info
            self.canvas.create_text(
                10, 10, 
                text=f"Frame {self.current_frame_index + 1}/{len(self.animation_frames)}", 
                anchor=tk.NW,
                fill="black"
            )
            
            # Show frame dimensions
            self.canvas.create_text(
                10, 30,
                text=f"Size: {frame.width}x{frame.height} px", 
                anchor=tk.NW,
                fill="black"
            )
            
            # Force canvas update to ensure image is displayed
            self.canvas.update()
            
        except Exception as e:
            import traceback
            traceback.print_exc()
            
            # Show error on canvas
            self.canvas.delete("all")
            self.canvas.create_text(
                self.canvas.winfo_width() // 2, 
                self.canvas.winfo_height() // 2,
                text=f"Error displaying frame: {str(e)}", 
                fill="red",
                font=("Arial", 12)
            )
    
    def next_frame(self):
        if self.animation_frames:
            self.current_frame_index = (self.current_frame_index + 1) % len(self.animation_frames)
            self.display_current_frame()
    
    def prev_frame(self):
        if self.animation_frames:
            self.current_frame_index = (self.current_frame_index - 1) % len(self.animation_frames)
            self.display_current_frame()
    
    def preview_contact_sheet(self):
        """
        Preview contact sheet with improved multi-page support and minimum frame sizes.
        """
        if not self.animation_frames:
            messagebox.showwarning("Warning", "No animation loaded")
            return
            
        # Create a preview of the contact sheet
        try:
            # Get settings
            frame_width = max(160, self.output_frame_width.get())  # Enforce minimum 160px width
            
            # Handle the frame height correctly - it's an integer, not an IntVar
            # Instead of using .get(), just use the value directly
            frame_height = self.output_frame_height  # This is already an integer
            
            frames_per_row = self.frames_per_row.get()
            reg_mark_size = self.registration_mark_size.get()
            output_dpi = self.output_dpi.get()
            
            # Ensure aspect ratio of frames is maintained if only width is specified
            if frame_height <= 0:
                # Get average aspect ratio from animation frames
                total_ratio = 0
                for frame in self.animation_frames:
                    if frame.height > 0:  # Avoid division by zero
                        total_ratio += frame.width / frame.height
                
                avg_ratio = total_ratio / len(self.animation_frames) if len(self.animation_frames) > 0 else 1.0
                frame_height = int(frame_width / avg_ratio)
            
            # Initialize the paper size handler
            paper_handler = PaperSizeHandler(
                paper_size=self.paper_size.get(),
                orientation=self.paper_orientation.get(),
                dpi=self.output_dpi.get()
            )

            
            # Calculate frames per page based on the minimum size
            padding = reg_mark_size * 2
            cell_width = frame_width + padding * 2
            cell_height = frame_height + padding * 2
            
            # Calculate frames per row and column that can fit on a page
            frames_per_row = min(frames_per_row, max(1, (paper_handler.width_px - 100) // cell_width))
            frames_per_col = max(1, (paper_handler.height_px - 100) // cell_height)
            frames_per_page = frames_per_row * frames_per_col
            
            # Calculate total number of pages needed
            num_frames = len(self.animation_frames)
            num_pages = math.ceil(num_frames / frames_per_page)
            
            # Create preview pages
            preview_pages = []
            
            # For each page, calculate which frames to include
            for page in range(num_pages):
                start_idx = page * frames_per_page
                end_idx = min(start_idx + frames_per_page, num_frames)
                
                # Create a new page
                page_img = Image.new('RGB', (paper_handler.width_px, paper_handler.height_px), (255, 255, 255))
                draw = ImageDraw.Draw(page_img)
                
                # Add page info
                if num_pages > 1:
                    draw.text((50, 20), f"Page {page+1} of {num_pages}", fill=(0, 0, 0))
                
                # Calculate grid width and height
                grid_width = frames_per_row * cell_width
                grid_height = min(frames_per_col, math.ceil((end_idx - start_idx) / frames_per_row)) * cell_height
                
                # Center the grid on the page
                x_offset = (paper_handler.width_px - grid_width) // 2
                y_offset = (paper_handler.height_px - grid_height) // 2
                
                # Place frames for this page
                for i in range(start_idx, end_idx):
                    local_idx = i - start_idx
                    row = local_idx // frames_per_row
                    col = local_idx % frames_per_row
                    
                    # Position for this frame
                    x = x_offset + col * cell_width + padding
                    y = y_offset + row * cell_height + padding
                    
                    # Resize frame to desired output size
                    resized_frame = self.animation_frames[i].resize((frame_width, frame_height), Image.LANCZOS)
                    
                    # Paste the frame into the contact sheet
                    page_img.paste(resized_frame, (x, y))
                    
                    # Add registration marks around each frame
                    mark_positions = [
                        (x - padding // 2, y - padding // 2),  # Top-left
                        (x + frame_width + padding // 2, y - padding // 2),  # Top-right
                        (x - padding // 2, y + frame_height + padding // 2),  # Bottom-left
                        (x + frame_width + padding // 2, y + frame_height + padding // 2)  # Bottom-right
                    ]
                    
                    # Draw registration marks
                    for mark_x, mark_y in mark_positions:
                        # Draw circle
                        circle_radius = reg_mark_size // 2
                        draw.ellipse(
                            (mark_x - circle_radius, mark_y - circle_radius, 
                            mark_x + circle_radius, mark_y + circle_radius),
                            outline=(0, 0, 0),
                            width=2
                        )
                        
                        # Draw crosshair
                        draw.line((mark_x - reg_mark_size, mark_y, mark_x + reg_mark_size, mark_y), fill=(0, 0, 0), width=2)
                        draw.line((mark_x, mark_y - reg_mark_size, mark_x, mark_y + reg_mark_size), fill=(0, 0, 0), width=2)
                        
                    # Add frame number
                    draw.text((x, y + frame_height + 5), f"Frame {i + 1}", fill=(0, 0, 0))
                
                preview_pages.append(page_img)
            
            if not preview_pages:
                messagebox.showwarning("Warning", "No pages generated in preview")
                return
            
            # Create a new window for preview
            preview_window = tk.Toplevel(self.root)
            preview_window.title("Contact Sheet Preview")
            preview_window.geometry("900x700")
            
            # Create a notebook for multi-page preview
            notebook = ttk.Notebook(preview_window)
            notebook.pack(fill=tk.BOTH, expand=True)
            
            # Store references to prevent garbage collection
            self.preview_photos = []
            
            # Add each page to the notebook
            for i, page in enumerate(preview_pages):
                # Create a frame for this page
                page_frame = ttk.Frame(notebook)
                notebook.add(page_frame, text=f"Page {i+1}")
                
                # Create a canvas with scrollbars
                page_canvas = tk.Canvas(page_frame, bg="white")
                h_scrollbar = ttk.Scrollbar(page_frame, orient=tk.HORIZONTAL, command=page_canvas.xview)
                v_scrollbar = ttk.Scrollbar(page_frame, orient=tk.VERTICAL, command=page_canvas.yview)
                
                page_canvas.configure(xscrollcommand=h_scrollbar.set, yscrollcommand=v_scrollbar.set)
                
                h_scrollbar.pack(side=tk.BOTTOM, fill=tk.X)
                v_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
                page_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
                
                # Resize the preview for display
                max_size = 800
                width, height = page.size
                
                if width > max_size or height > max_size:
                    ratio = width / height
                    if width > height:
                        new_width = max_size
                        new_height = int(max_size / ratio)
                    else:
                        new_height = max_size
                        new_width = int(max_size * ratio)
                        
                    preview_img = page.resize((new_width, new_height), Image.LANCZOS)
                else:
                    preview_img = page
                
                # Display the preview
                preview_photo = ImageTk.PhotoImage(preview_img)
                self.preview_photos.append(preview_photo)  # Keep a reference to prevent garbage collection
                
                canvas_item = page_canvas.create_image(0, 0, image=preview_photo, anchor=tk.NW)
                
                # Configure scrollable area
                page_canvas.configure(scrollregion=(0, 0, preview_img.width, preview_img.height))
                
                # Add information about layout
                if i == 0:  # Only on first page
                    info_text = (f"Layout: {frames_per_row} frames per row, {frames_per_col} rows per page\n"
                                f"Frame size: {frame_width}x{frame_height} pixels")
                    info_label = ttk.Label(page_frame, text=info_text)
                    info_label.pack(side=tk.BOTTOM, fill=tk.X)
            
        except Exception as e:
            import traceback
            traceback.print_exc()
            messagebox.showerror("Error", f"Failed to create preview: {str(e)}")

    # Add this to your generate_contact_sheet method, replacing the equivalent section:
    # Add this method to the AnimationToContactSheetApp class in contact_sheet.ipynb
    def generate_contact_sheet(self):
        """
        Generate a contact sheet from animation frames with improved registration marks.
        Uses auto-calculated frame height based on aspect ratio.
        """
        if not self.animation_frames:
            messagebox.showwarning("Warning", "No animation loaded")
            return
        
        try:
            # Ask user for output directory
            output_dir = filedialog.askdirectory(
                title="Select Output Directory",
                initialdir=os.path.expanduser("~")
            )
            
            if not output_dir:
                return  # User cancelled
            
            # Get settings
            options = {
                "frame_width": max(200, self.output_frame_width.get()),
                "frames_per_row": self.frames_per_row.get(),
                "reg_mark_size": self.registration_mark_size.get(),
                "output_dpi": self.output_dpi.get(),
                "paper_size": self.paper_size.get(),
                "orientation": self.paper_orientation.get(),
                "cmyk_separation": self.cmyk_separation.get(),
                "add_frame_numbers": True,
                "border_width": 2,
                "spacing": 10  # Add spacing between frames for clarity
            }
            
            # Update status
            self.status_var.set(f"Generating contact sheet with frame size: {options['frame_width']}px...")
            self.root.update()
            
            # Create timestamp for unique filenames
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            base_filename = os.path.splitext(os.path.basename(self.animation_path))[0] if self.animation_path else "animation"
            output_base = f"{base_filename}_contact_sheet_{timestamp}"
            
            # Calculate frame height based on aspect ratio
            frame_height = self.calculate_frame_height()
            
            # Paper dimensions in pixels based on DPI
            paper_dimensions = {
                "letter": (8.5 * 72, 11 * 72),  # 8.5" x 11"
                "tabloid": (11 * 72, 17 * 72),  # 11" x 17"
                "a4": (8.27 * 72, 11.69 * 72),  # 210mm x 297mm
                "a3": (11.69 * 72, 16.54 * 72)  # 297mm x 420mm
            }
            
            # Get paper dimensions
            width_pts, height_pts = paper_dimensions.get(options["paper_size"], paper_dimensions["letter"])
            
            # Apply orientation
            if options["orientation"] == "landscape":
                width_pts, height_pts = height_pts, width_pts
            
            # Convert points to pixels at specified DPI
            paper_width = int(width_pts / 72 * options["output_dpi"])
            paper_height = int(height_pts / 72 * options["output_dpi"])
            
            # Calculate padding based on registration marks and frame numbers
            padding = options["reg_mark_size"] + options["spacing"]
            
            # Calculate cell dimensions (includes padding)
            cell_width = options["frame_width"] + (padding * 2)
            cell_height = frame_height + (padding * 2) + (30 if options["add_frame_numbers"] else 0)
            
            # Calculate frames per row and column that can fit on a page
            frames_per_row = min(options["frames_per_row"], max(1, (paper_width - 100) // cell_width))
            frames_per_col = max(1, (paper_height - 100) // cell_height)
            frames_per_page = frames_per_row * frames_per_col
            
            # Calculate total number of pages needed
            num_frames = len(self.animation_frames)
            num_pages = math.ceil(num_frames / frames_per_page)
            
            # Skip auto-multipage if disabled and show warning if needed
            if not self.auto_multipage.get() and num_pages > 1:
                result = messagebox.askyesno(
                    "Multiple Pages Required",
                    f"Your animation requires {num_pages} pages at the current size settings.\n\n"
                    f"Would you like to proceed with multiple pages?"
                )
                if not result:
                    self.status_var.set("Contact sheet generation cancelled")
                    return
            
            # Function to draw improved registration marks
            def draw_registration_marks(draw, x, y, size, color):
                """Draw improved registration marks at the specified position"""
                # Draw circle
                circle_radius = size // 2
                draw.ellipse(
                    (x - circle_radius, y - circle_radius, x + circle_radius, y + circle_radius),
                    outline=color,
                    width=2
                )
                
                # Draw crosshair
                draw.line((x - size, y, x + size, y), fill=color, width=2)
                draw.line((x, y - size, x, y + size), fill=color, width=2)
            
            # Calculate grid dimensions
            grid_width = frames_per_row * cell_width
            grid_height = min(frames_per_col, math.ceil(num_frames / frames_per_row)) * cell_height
            
            # Calculate centering offsets
            x_offset = (paper_width - grid_width) // 2
            y_offset = (paper_height - grid_height) // 2
            
            # Inform user about multi-page output if applicable
            if num_pages > 1:
                self.status_var.set(f"Creating {num_pages} pages to maintain frame size of {options['frame_width']}x{frame_height}px...")
                self.root.update()
            
            # Create the contact sheets
            if options["cmyk_separation"]:
                # Create separate CMYK plates
                channels = ['C', 'M', 'Y', 'K']
                
                for i, channel in enumerate(channels):
                    # Prepare all pages for this CMYK channel
                    cmyk_pages = []
                    
                    for page in range(num_pages):
                        # Create a blank sheet - USE WHITE (255) as background for CMYK plates
                        sheet = Image.new('L', (paper_width, paper_height), 255)
                        draw = ImageDraw.Draw(sheet)
                        
                        # Add page info at the top
                        if num_pages > 1:
                            draw.text((50, 20), f"Page {page+1} of {num_pages} - {channel} plate", fill=0)
                        
                        # Calculate start and end frame indices for this page
                        start_idx = page * frames_per_page
                        end_idx = min(start_idx + frames_per_page, num_frames)
                        
                        # Place frames in a grid
                        for j in range(start_idx, end_idx):
                            local_idx = j - start_idx
                            row = local_idx // frames_per_row
                            col = local_idx % frames_per_row
                            
                            # Calculate cell position
                            cell_x = x_offset + col * cell_width
                            cell_y = y_offset + row * cell_height
                            
                            # Calculate frame position within cell
                            frame_x = cell_x + padding
                            frame_y = cell_y + padding
                            
                            # Resize and prepare frame
                            frame_rgb = self.animation_frames[j].convert('RGB')
                            resized_frame = frame_rgb.resize((options["frame_width"], frame_height), Image.LANCZOS)
                            
                            # Add black border for visibility
                            if options["border_width"] > 0:
                                resized_frame = ImageOps.expand(resized_frame, border=options["border_width"], fill='black')
                            
                            # Convert to CMYK and extract channel
                            cmyk_frame = resized_frame.convert('CMYK')
                            channel_frame = cmyk_frame.split()[i]
                            
                            # Invert the channel (in CMYK, 0 is full color and 255 is no color)
                            inverted_channel = Image.eval(channel_frame, lambda x: 255 - x)
                            
                            # Paste the channel into the contact sheet
                            sheet.paste(inverted_channel, (frame_x, frame_y))
                            
                            # Add registration marks at all four corners of the frame
                            frame_width_with_border = options["frame_width"] + (options["border_width"] * 2)
                            frame_height_with_border = frame_height + (options["border_width"] * 2)
                            
                            # Define corner positions
                            corners = [
                                (frame_x - padding//2, frame_y - padding//2),                              # Top-left
                                (frame_x + frame_width_with_border + padding//2, frame_y - padding//2),    # Top-right
                                (frame_x - padding//2, frame_y + frame_height_with_border + padding//2),   # Bottom-left
                                (frame_x + frame_width_with_border + padding//2, frame_y + frame_height_with_border + padding//2)  # Bottom-right
                            ]
                            
                            # Draw registration marks at each corner
                            for corner_x, corner_y in corners:
                                draw_registration_marks(draw, corner_x, corner_y, options["reg_mark_size"], 0)
                            
                            # Add frame number
                            if options["add_frame_numbers"]:
                                draw.text(
                                    (frame_x, frame_y + frame_height_with_border + 5),
                                    f"Frame {j + 1}",
                                    fill=0
                                )
                        
                        # Add this page to the list
                        cmyk_pages.append(sheet)
                    
                    # Save all pages as a multi-page PDF
                    output_path = os.path.join(output_dir, f"{output_base}_{channel}.pdf")
                    
                    # Check if we have multiple pages
                    if len(cmyk_pages) > 1:
                        # Save first page, then append the rest
                        cmyk_pages[0].save(
                            output_path, 
                            "PDF", 
                            resolution=options["output_dpi"],
                            save_all=True, 
                            append_images=cmyk_pages[1:]
                        )
                    else:
                        # Just save a single page
                        cmyk_pages[0].save(output_path, "PDF", resolution=options["output_dpi"])
                    
                    self.status_var.set(f"Saved {channel} plate ({i+1}/4)...")
                    self.root.update()
                
                messagebox.showinfo("Success", f"Generated {num_pages}-page CMYK separated contact sheets in {output_dir}")
                
            else:
                # Create RGB contact sheet pages
                rgb_pages = []
                
                for page in range(num_pages):
                    self.status_var.set(f"Creating page {page+1} of {num_pages}...")
                    self.root.update()
                    
                    sheet = Image.new('RGB', (paper_width, paper_height), (255, 255, 255))
                    draw = ImageDraw.Draw(sheet)
                    
                    # Add page info at the top
                    if num_pages > 1:
                        draw.text((50, 20), f"Page {page+1} of {num_pages}", fill=(0, 0, 0))
                    
                    # Calculate start and end frame indices for this page
                    start_idx = page * frames_per_page
                    end_idx = min(start_idx + frames_per_page, num_frames)
                    
                    # Place frames in a grid
                    for j in range(start_idx, end_idx):
                        local_idx = j - start_idx
                        row = local_idx // frames_per_row
                        col = local_idx % frames_per_row
                        
                        # Calculate cell position
                        cell_x = x_offset + col * cell_width
                        cell_y = y_offset + row * cell_height
                        
                        # Calculate frame position within cell
                        frame_x = cell_x + padding
                        frame_y = cell_y + padding
                        
                        # Resize and prepare frame
                        frame_rgb = self.animation_frames[j].convert('RGB')
                        resized_frame = frame_rgb.resize((options["frame_width"], frame_height), Image.LANCZOS)
                        
                        # Add black border for visibility
                        if options["border_width"] > 0:
                            resized_frame = ImageOps.expand(resized_frame, border=options["border_width"], fill='black')
                        
                        # Paste the frame into the contact sheet
                        sheet.paste(resized_frame, (frame_x, frame_y))
                        
                        # Add registration marks at all four corners of the frame
                        frame_width_with_border = options["frame_width"] + (options["border_width"] * 2)
                        frame_height_with_border = frame_height + (options["border_width"] * 2)
                        
                        # Define corner positions
                        corners = [
                            (frame_x - padding//2, frame_y - padding//2),                              # Top-left
                            (frame_x + frame_width_with_border + padding//2, frame_y - padding//2),    # Top-right
                            (frame_x - padding//2, frame_y + frame_height_with_border + padding//2),   # Bottom-left
                            (frame_x + frame_width_with_border + padding//2, frame_y + frame_height_with_border + padding//2)  # Bottom-right
                        ]
                        
                        # Draw registration marks at each corner
                        for corner_x, corner_y in corners:
                            draw_registration_marks(draw, corner_x, corner_y, options["reg_mark_size"], (0, 0, 0))
                        
                        # Add frame number
                        if options["add_frame_numbers"]:
                            draw.text(
                                (frame_x, frame_y + frame_height_with_border + 5),
                                f"Frame {j + 1}",
                                fill=(0, 0, 0)
                            )
                    
                    # Add this page to the list
                    rgb_pages.append(sheet)
                    
                    # Also save individual PNGs for each page if requested
                    if num_pages > 1:
                        png_path = os.path.join(output_dir, f"{output_base}_page{page+1}.png")
                        sheet.save(png_path, "PNG")
                
                # Save the multi-page PDF
                pdf_path = os.path.join(output_dir, f"{output_base}.pdf")
                
                self.status_var.set(f"Saving PDF with {num_pages} pages...")
                self.root.update()
                
                # Check if we have multiple pages
                if len(rgb_pages) > 1:
                    # Save first page, then append the rest
                    rgb_pages[0].save(
                        pdf_path, 
                        "PDF", 
                        resolution=options["output_dpi"],
                        save_all=True, 
                        append_images=rgb_pages[1:]
                    )
                else:
                    # Just save a single page
                    rgb_pages[0].save(pdf_path, "PDF", resolution=options["output_dpi"])
                
                # Save a PNG of the first page for quick preview
                png_path = os.path.join(output_dir, f"{output_base}.png")
                rgb_pages[0].save(png_path, "PNG")
                
                self.status_var.set(f"Contact sheets saved to {output_dir}")
                messagebox.showinfo("Success", f"Generated {num_pages}-page contact sheet in {output_dir}")
                
        except Exception as e:
            import traceback
            traceback.print_exc()
            self.status_var.set(f"Error: {str(e)}")
            messagebox.showerror("Error", f"Failed to generate contact sheet: {str(e)}")


class PaperSizeHandler:
    """
    Utility class to handle standard paper sizes for contact sheets.
    Provides functionality for creating multi-page contact sheets based on standard paper sizes.
    """
    
    # Standard paper sizes in points (72 points = 1 inch)
    # Using slightly smaller dimensions to account for printer margins
    PAPER_SIZES = {
        "letter": (7.5 * 72, 10 * 72),       # 8.5" x 11" with 0.5" margins
        "tabloid": (10.5 * 72, 16.5 * 72),   # 11" x 17" with 0.5" margins
        "a4": (7.7 * 72, 10.7 * 72),         # 210mm x 297mm with 12.7mm margins
        "a3": (11.2 * 72, 15.9 * 72)         # 297mm x 420mm with 12.7mm margins
    }
    
    def __init__(self, paper_size="letter", orientation="portrait", dpi=300, margin_mm=12.7):
        """
        Initialize the paper size handler.
        
        Args:
            paper_size: The paper size to use ("letter", "tabloid", "a4", "a3")
            orientation: Paper orientation ("portrait" or "landscape")
            dpi: Output resolution in dots per inch
            margin_mm: Margin in millimeters
        """
        if paper_size not in self.PAPER_SIZES:
            raise ValueError(f"Unsupported paper size: {paper_size}")
        
        # Get the base paper dimensions
        width_pts, height_pts = self.PAPER_SIZES[paper_size]
        
        # Apply orientation
        if orientation == "landscape":
            width_pts, height_pts = height_pts, width_pts
            
        # Convert points to pixels at the specified DPI
        self.width_px = int(width_pts / 72 * dpi)
        self.height_px = int(height_pts / 72 * dpi)
        
        self.paper_size = paper_size
        self.orientation = orientation
        self.dpi = dpi
        
    def calculate_frames_per_page(self, frame_width, frame_height, padding):
        """
        Calculate how many frames can fit on a single page.
        
        Args:
            frame_width: Width of each frame in pixels
            frame_height: Height of each frame in pixels
            padding: Padding around each frame in pixels
            
        Returns:
            (frames_per_row, rows_per_page)
        """
        # Calculate available space
        total_frame_width = frame_width + (padding * 2)
        total_frame_height = frame_height + (padding * 2)
        
        # Calculate frames that can fit
        frames_per_row = max(1, math.floor(self.width_px / total_frame_width))
        rows_per_page = max(1, math.floor(self.height_px / total_frame_height))
        
        return frames_per_row, rows_per_page
            
    def create_multi_page_contact_sheet(self, frames, frame_width, frame_height, reg_mark_size, 
                                    cmyk_channel=None, font=None, add_frame_numbers=True):
        """
        Create a multi-page contact sheet with improved registration marks.
        
        Args:
            frames: List of frame images
            frame_width: Width of each frame in the contact sheet
            frame_height: Height of each frame in the contact sheet
            reg_mark_size: Size of registration marks
            cmyk_channel: CMYK channel to extract (0-3 for C,M,Y,K) or None for RGB
            font: Font to use for frame numbers
            add_frame_numbers: Whether to add frame numbers
            
        Returns:
            List of PIL.Image objects, each representing a page
        """
        # Calculate padding based on registration marks and frame numbers
        padding = reg_mark_size * 2 + (20 if add_frame_numbers else 0)
        
        # Calculate frames per page
        frames_per_row, rows_per_page = self.calculate_frames_per_page(frame_width, frame_height, padding)
        frames_per_page = frames_per_row * rows_per_page
        
        # Calculate number of pages needed
        num_frames = len(frames)
        num_pages = math.ceil(num_frames / frames_per_page)
        
        # Create pages
        pages = []
        
        for page_idx in range(num_pages):
            # Create a blank page
            if cmyk_channel is not None:
                # Create a grayscale image for CMYK separation
                page = Image.new('L', (self.width_px, self.height_px), 255)
            else:
                # Create RGB image
                page = Image.new('RGB', (self.width_px, self.height_px), (255, 255, 255))
                
            draw = ImageDraw.Draw(page)
            
            # Add page number at the top
            if font:
                page_text = f"Page {page_idx + 1} of {num_pages}"
                text_width = draw.textlength(page_text, font=font)
                draw.text(
                    (self.width_px // 2 - text_width // 2, 10), 
                    page_text, 
                    fill=0 if cmyk_channel is not None else (0, 0, 0), 
                    font=font
                )
            
            # Calculate start and end frame indices for this page
            start_idx = page_idx * frames_per_page
            end_idx = min(start_idx + frames_per_page, num_frames)
            
            # Place frames on this page
            for i in range(start_idx, end_idx):
                frame_idx = i - start_idx
                
                # Calculate position in the grid
                row = frame_idx // frames_per_row
                col = frame_idx % frames_per_row
                
                # Calculate frame position (center of cell)
                cell_width = frame_width + padding * 2
                cell_height = frame_height + padding * 2
                
                # Position of cell's top-left corner
                cell_x = col * cell_width
                cell_y = row * cell_height + 30  # Add 30px for page header
                
                # Frame position (within the cell, leaving space for registration marks)
                frame_x = cell_x + padding
                frame_y = cell_y + padding
                
                # Resize frame to desired output size
                frame = frames[i]
                resized_frame = frame.resize((frame_width, frame_height), Image.LANCZOS)
                
                # If CMYK separation is enabled and a specific channel is requested
                if cmyk_channel is not None:
                    # Convert to CMYK
                    cmyk_frame = resized_frame.convert('CMYK')
                    # Extract the specified channel
                    channel_frame = cmyk_frame.split()[cmyk_channel]
                    # Paste the channel into the contact sheet
                    page.paste(channel_frame, (frame_x, frame_y))
                else:
                    # RGB mode - paste the frame directly
                    page.paste(resized_frame, (frame_x, frame_y))
                
                # Draw improved registration marks at all four corners
                reg_color = 0 if cmyk_channel is not None else (0, 0, 0)
                line_thickness = 2
                
                # Define corner positions
                corners = [
                    # Top-left
                    (cell_x, cell_y),
                    # Top-right
                    (cell_x + cell_width, cell_y),
                    # Bottom-left
                    (cell_x, cell_y + cell_height),
                    # Bottom-right
                    (cell_x + cell_width, cell_y + cell_height)
                ]
                
                # Draw registration marks at each corner, ensuring they're inside the page bounds
                for corner_x, corner_y in corners:
                    # Ensure the corner coordinates are valid integers
                    corner_x = int(corner_x)
                    corner_y = int(corner_y)
                    
                    # Make sure registration marks are fully inside the page
                    mark_radius = reg_mark_size // 2
                    
                    # Ensure mark coordinates are valid
                    mark_left = max(5, corner_x - mark_radius)
                    mark_right = min(self.width_px - 5, corner_x + mark_radius)
                    mark_top = max(5, corner_y - mark_radius)
                    mark_bottom = min(self.height_px - 5, corner_y + mark_radius)
                    
                    # Ensure right > left and bottom > top
                    if mark_right <= mark_left:
                        mark_right = mark_left + 1
                    if mark_bottom <= mark_top:
                        mark_bottom = mark_top + 1
                    
                    # Draw circle - ensure fully within bounds
                    draw.ellipse(
                        (mark_left, mark_top, mark_right, mark_bottom),
                        outline=reg_color,
                        width=line_thickness
                    )
                    
                    # Calculate crosshair line extents to ensure they're in bounds
                    h_line_left = max(5, corner_x - reg_mark_size)
                    h_line_right = min(self.width_px - 5, corner_x + reg_mark_size)
                    v_line_top = max(5, corner_y - reg_mark_size)
                    v_line_bottom = min(self.height_px - 5, corner_y + reg_mark_size)
                    
                    # Draw crosshair (horizontal line)
                    if h_line_left < h_line_right:
                        draw.line(
                            (h_line_left, corner_y, h_line_right, corner_y),
                            fill=reg_color, 
                            width=line_thickness
                        )
                    
                    # Draw crosshair (vertical line)
                    if v_line_top < v_line_bottom:
                        draw.line(
                            (corner_x, v_line_top, corner_x, v_line_bottom),
                            fill=reg_color, 
                            width=line_thickness
                        )
                
                # Add frame number below the frame
                if add_frame_numbers and font:
                    number_text = f"F {i + 1}"
                    text_width = draw.textlength(number_text, font=font)
                    draw.text(
                        (frame_x + (frame_width - text_width) // 2, frame_y + frame_height + 5), 
                        number_text, 
                        fill=reg_color, 
                        font=font
                    )
            
            pages.append(page)
        
        return pages

if __name__ == "__main__":
    root = tk.Tk()
    app = AnimationToContactSheetApp(root)
    root.mainloop()

2025-03-25 21:11:23.857 Python[22929:1275982] +[IMKClient subclass]: chose IMKClient_Legacy
2025-03-25 21:11:23.857 Python[22929:1275982] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
